In [1]:
import numpy as np
import sncosmo
import matplotlib.pyplot as plt
from astropy import units as u
from astropy.cosmology import WMAP9 as cosmo
from astropy import constants as const
import sys
sys.path.append('../')
from scripts import tde_utils

In [2]:
import seaborn as sns
sns.set_context('paper')

In [3]:
jwst_filters =['f070w', 
               'f115w', 
               'f150w',
               'f277w',
               'f356w',
               'f444w',
               'f140m',
               'f182m',
               'f250m',
               'f335m',
               'f410m',
               'f480m'
              ]

In [4]:
def d_maglim(z, args):
    try:
        filt, maglim = args
        source = tde_utils.BlackBodySource(30000)
        bb_model = sncosmo.Model(source=source)
        bb_model.set(z=z)
        luminosity_distance = cosmo.luminosity_distance(z)
        
        filter_abs_mag = bb_model.bandmag(filt, 'ab', 0)
        app_mag = filter_abs_mag + 5*np.log10(luminosity_distance/(10 * u.parsec))
        #print(np.abs(app_mag - (maglim-1)))
        return np.abs(app_mag - (maglim))
    except:
        return 1000

In [5]:
from scipy.optimize import minimize

In [41]:
maglim = 26.7

z_lims = {}
for filt in jwst_filters:
    res = minimize(d_maglim, 
                   args = [filt, maglim], 
                   bounds = [[0.01, 25]],
                   x0=2)
    z_lims[filt] = res.x[0]

In [44]:
z_lims

{'f070w': 5.7668604204152123,
 'f115w': 7.2787094606093445,
 'f150w': 7.5889344005275774,
 'f277w': 2.9353922533228864,
 'f356w': 1.4576834828819554,
 'f444w': 0.94442263218006994,
 'f140m': 7.5997848533668089,
 'f182m': 7.1725675716063524,
 'f250m': 4.1343816386805852,
 'f335m': 1.6453480769658038,
 'f410m': 1.0852399486154769,
 'f480m': 0.81178253488624219}

In [45]:
## TDE Rate: 5×10−8/𝑦𝑟/𝑀𝑝𝑐3
tde_rate = (5e-8 * u.year**(-1) * u.Mpc**(-3))

snia_rate = 2.4e-5 * u.year**(-1) * u.Mpc**(-3) # https://arxiv.org/abs/1903.08580

In [46]:
def calc_volume(sq_degrees, z):
    
    sphere_area_sterad = 4*np.pi
    sphere_area_degrees = sphere_area_sterad * ((180/np.pi)**2)
    fractional_area = sq_degrees / sphere_area_degrees

    total_volume = cosmo.comoving_volume(z)
    
    return total_volume * fractional_area

In [47]:
sq_degs = 1
filter_volumes = {filt: calc_volume(sq_degs, filtz) for filt, filtz in z_lims.items()}

In [48]:
filter_volumes

{'f070w': <Quantity 59149018.77174914 Mpc3>,
 'f115w': <Quantity 72672197.18354681 Mpc3>,
 'f150w': <Quantity 75187171.9742035 Mpc3>,
 'f277w': <Quantity 27109536.65247382 Mpc3>,
 'f356w': <Quantity 8477006.615104115 Mpc3>,
 'f444w': <Quantity 3397037.3904661345 Mpc3>,
 'f140m': <Quantity 75273677.16089864 Mpc3>,
 'f182m': <Quantity 71792797.47990133 Mpc3>,
 'f250m': <Quantity 41794603.17236377 Mpc3>,
 'f335m': <Quantity 10660771.492272481 Mpc3>,
 'f410m': <Quantity 4622233.783187552 Mpc3>,
 'f480m': <Quantity 2393392.4879991803 Mpc3>}

In [49]:
{filt: (tde_rate * calc_volume(sq_degs, filtz)).value for filt, filtz in z_lims.items()}

{'f070w': 2.9574509385874568,
 'f115w': 3.6336098591773403,
 'f150w': 3.7593585987101745,
 'f277w': 1.3554768326236908,
 'f356w': 0.42385033075520573,
 'f444w': 0.16985186952330672,
 'f140m': 3.7636838580449319,
 'f182m': 3.5896398739950661,
 'f250m': 2.0897301586181887,
 'f335m': 0.53303857461362403,
 'f410m': 0.2311116891593776,
 'f480m': 0.11966962439995901}

In [36]:
0.15*10000

1500.0